This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

# Fundamentals of machine learning

## Generalization: The goal of machine learning

### Underfitting and overfitting

#### Noisy training data

#### Ambiguous features

#### Rare features and spurious correlations

**Adding white-noise channels or all-zeros channels to MNIST**

In [ ]:
from tensorflow.keras.datasets import mnist # importing the mnist dataset from keras library 
import numpy as np # importing numpy library to work with arrays 

(train_images, train_labels), _ = mnist.load_data() # loading the mnist dataset into train_images and train_labels 
train_images = train_images.reshape((60000, 28 * 28)) # reshaping the train_images to 60000, 28*28 
train_images = train_images.astype("float32") / 255 # normalizing the train_images by dividing by 255 

train_images_with_noise_channels = np.concatenate( # adding random noise to the train_images 
    [train_images, np.random.random((len(train_images), 784))], axis=1) # adding random noise to the train_images 

train_images_with_zeros_channels = np.concatenate( # adding zeros to the train_images
    [train_images, np.zeros((len(train_images), 784))], axis=1) # adding zeros to the train_images

**Training the same model on MNIST data with noise channels or all-zero channels**

In [ ]:
from tensorflow import keras # importing keras from tensorflow library
from tensorflow.keras import layers # importing layers from keras library

def get_model(): # defining a function to get the model
    model = keras.Sequential([ # creating a sequential model
        layers.Dense(512, activation="relu"), # adding a dense layer with 512 neurons and relu activation function
        layers.Dense(10, activation="softmax") # adding a dense layer with 10 neurons and softmax activation function
    ])
    model.compile(optimizer="rmsprop", # compiling the model with optimizer as rmsprop
                  loss="sparse_categorical_crossentropy", # setting loss function as sparse_categorical_crossentropy
                  metrics=["accuracy"]) # setting the metrics as accuracy
    return model # returning the model

model = get_model() # getting the model
history_noise = model.fit( # fitting the model with train_images_with_noise_channels and train_labels
    train_images_with_noise_channels, train_labels, 
    epochs=10, # setting the epochs as 10
    batch_size=128, # setting the batch size as 128
    validation_split=0.2) # setting the validation split as 0.2

model = get_model() # getting the model
history_zeros = model.fit( # fitting the model with train_images_with_zeros_channels and train_labels
    train_images_with_zeros_channels, train_labels,
    epochs=10, # setting the epochs as 10
    batch_size=128, # setting the batch size as 128
    validation_split=0.2) # setting the validation split as 0.2

**Plotting a validation accuracy comparison**

In [ ]:
import matplotlib.pyplot as plt # importing matplotlib library to plot the graph
val_acc_noise = history_noise.history["val_accuracy"] # getting the validation accuracy from history_noise
val_acc_zeros = history_zeros.history["val_accuracy"] # getting the validation accuracy from history_zeros
epochs = range(1, 11) # setting the epochs as 1 to 11
plt.plot(epochs, val_acc_noise, "b-", # plotting the graph with epochs, val_acc_noise and val_acc_zeros
         label="Validation accuracy with noise channels") # setting the label as Validation accuracy with noise channels
plt.plot(epochs, val_acc_zeros, "b--", # plotting the graph with epochs, val_acc_noise and val_acc_zeros
         label="Validation accuracy with zeros channels") # setting the label as Validation accuracy with zeros channels
plt.title("Effect of noise channels on validation accuracy") # setting the title of the graph
plt.xlabel("Epochs") # setting the x-axis label as Epochs
plt.ylabel("Accuracy") # setting the y-axis label as Accuracy
plt.legend() # setting the legend 

### The nature of generalization in deep learning

**Fitting a MNIST model with randomly shuffled labels**

In [ ]:
(train_images, train_labels), _ = mnist.load_data() # loading the mnist dataset into train_images and train_labels
train_images = train_images.reshape((60000, 28 * 28)) # reshaping the train_images to 60000, 28*28
train_images = train_images.astype("float32") / 255 # normalizing the train_images by dividing by 255

random_train_labels = train_labels[:] # copying the train_labels to random_train_labels
np.random.shuffle(random_train_labels) # shuffling the random_train_labels

model = keras.Sequential([ # creating a sequential model
    layers.Dense(512, activation="relu"), # adding a dense layer with 512 neurons and relu activation function
    layers.Dense(10, activation="softmax") # adding a dense layer with 10 neurons and softmax activation function
])
model.compile(optimizer="rmsprop", # compiling the model with optimizer as rmsprop
              loss="sparse_categorical_crossentropy", # setting loss function as sparse_categorical_crossentropy
              metrics=["accuracy"]) # setting the metrics as accuracy
model.fit(train_images, random_train_labels, # fitting the model with train_images and random_train_labels
          epochs=100, # setting the epochs as 100
          batch_size=128, # setting the batch size as 128
          validation_split=0.2) # setting the validation split as 0.2

#### The manifold hypothesis

#### Interpolation as a source of generalization

#### Why deep learning works

#### Training data is paramount

## Evaluating machine-learning models

### Training, validation, and test sets

#### Simple hold-out validation

#### K-fold validation

#### Iterated K-fold validation with shuffling

### Beating a common-sense baseline

### Things to keep in mind about model evaluation

## Improving model fit

### Tuning key gradient descent parameters

**Training a MNIST model with an incorrectly high learning rate**

In [ ]:
(train_images, train_labels), _ = mnist.load_data() # loading the mnist dataset into train_images and train_labels
train_images = train_images.reshape((60000, 28 * 28)) # reshaping the train_images to 60000, 28*28
train_images = train_images.astype("float32") / 255 # normalizing the train_images by dividing by 255

model = keras.Sequential([ # creating a sequential model
    layers.Dense(512, activation="relu"), # adding a dense layer with 512 neurons and relu activation function
    layers.Dense(10, activation="softmax") # adding a dense layer with 10 neurons and softmax activation function
])
model.compile(optimizer=keras.optimizers.RMSprop(1.), # compiling the model with optimizer as RMSprop with learning rate as 1.0
              loss="sparse_categorical_crossentropy", # setting loss function as sparse_categorical_crossentropy
              metrics=["accuracy"]) # setting the metrics as accuracy
model.fit(train_images, train_labels, # fitting the model with train_images and train_labels
          epochs=10, # setting the epochs as 10
          batch_size=128, # setting the batch size as 128
          validation_split=0.2) # setting the validation split as 0.2

**The same model with a more appropriate learning rate**

In [ ]:
model = keras.Sequential([ # creating a sequential model
    layers.Dense(512, activation="relu"), # adding a dense layer with 512 neurons and relu activation function
    layers.Dense(10, activation="softmax") # adding a dense layer with 10 neurons and softmax activation function
])
model.compile(optimizer=keras.optimizers.RMSprop(1e-2), # compiling the model with optimizer as RMSprop with learning rate as 0.01
              loss="sparse_categorical_crossentropy", # setting loss function as sparse_categorical_crossentropy
              metrics=["accuracy"]) # setting the metrics as accuracy
model.fit(train_images, train_labels, # fitting the model with train_images and train_labels
          epochs=10, # setting the epochs as 10
          batch_size=128, # setting the batch size as 128
          validation_split=0.2) # setting the validation split as 0.2

### Leveraging better architecture priors

### Increasing model capacity

**A simple logistic regression on MNIST**

In [ ]:
model = keras.Sequential([layers.Dense(10, activation="softmax")]) # creating a sequential model with a dense layer with 10 neurons and softmax activation function
model.compile(optimizer="rmsprop", # compiling the model with optimizer as rmsprop
              loss="sparse_categorical_crossentropy", # setting loss function as sparse_categorical_crossentropy
              metrics=["accuracy"]) # setting the metrics as accuracy
history_small_model = model.fit( # fitting the model with train_images and train_labels
    train_images, train_labels, 
    epochs=20, # setting the epochs as 20
    batch_size=128, # setting the batch size as 128
    validation_split=0.2) # setting the validation split as 0.2

In [ ]:
import matplotlib.pyplot as plt # importing matplotlib library to plot the graph
val_loss = history_small_model.history["val_loss"] # getting the validation loss from history_small_model
epochs = range(1, 21) # setting the epochs as 1 to 21
plt.plot(epochs, val_loss, "b--", # plotting the graph with epochs and val_loss
         label="Validation loss") # setting the label as Validation loss
plt.title("Effect of insufficient model capacity on validation loss") # setting the title of the graph
plt.xlabel("Epochs") # setting the x-axis label as Epochs
plt.ylabel("Loss") # setting the y-axis label as Loss
plt.legend() # setting the legend

In [ ]:
model = keras.Sequential([ # creating a sequential model
    layers.Dense(96, activation="relu"), # adding a dense layer with 96 neurons and relu activation function
    layers.Dense(96, activation="relu"), # adding a dense layer with 96 neurons and relu activation function
    layers.Dense(10, activation="softmax"), # adding a dense layer with 10 neurons and softmax activation function
])
model.compile(optimizer="rmsprop", # compiling the model with optimizer as rmsprop
              loss="sparse_categorical_crossentropy", # setting loss function as sparse_categorical_crossentropy
              metrics=["accuracy"]) # setting the metrics as accuracy
history_large_model = model.fit( # fitting the model with train_images and train_labels
    train_images, train_labels,
    epochs=20, # setting the epochs as 20
    batch_size=128, # setting the batch size as 128
    validation_split=0.2) # setting the validation split as 0.2

## Improving generalization

### Dataset curation

### Feature engineering

### Using early stopping

### Regularizing your model

#### Reducing the network's size

**Original model**

In [ ]:
from tensorflow.keras.datasets import imdb # importing the imdb dataset from keras library
(train_data, train_labels), _ = imdb.load_data(num_words=10000) # loading the imdb dataset into train_data and train_labels with 10000 words

def vectorize_sequences(sequences, dimension=10000): # defining a function to vectorize the sequences
    results = np.zeros((len(sequences), dimension)) # creating a results array with zeros
    for i, sequence in enumerate(sequences): # iterating through the sequences
        results[i, sequence] = 1. # setting the results[i, sequence] as 1
    return results # returning the results
train_data = vectorize_sequences(train_data) # vectorizing the train_data

model = keras.Sequential([ # creating a sequential model
    layers.Dense(16, activation="relu"), # adding a dense layer with 16 neurons and relu activation function
    layers.Dense(16, activation="relu"), # adding a dense layer with 16 neurons and relu activation function
    layers.Dense(1, activation="sigmoid") # adding a dense layer with 1 neuron and sigmoid activation function
])
model.compile(optimizer="rmsprop", # compiling the model with optimizer as rmsprop
              loss="binary_crossentropy", # setting loss function as binary_crossentropy
              metrics=["accuracy"]) # setting the metrics as accuracy
history_original = model.fit(train_data, train_labels, # fitting the model with train_data and train_labels
                             epochs=20, batch_size=512, validation_split=0.4) # setting the epochs as 20, batch size as 512 and validation split as 0.4

**Version of the model with lower capacity**

In [ ]:
model = keras.Sequential([ # creating a sequential model
    layers.Dense(4, activation="relu"), # adding a dense layer with 4 neurons and relu activation function
    layers.Dense(4, activation="relu"), # adding a dense layer with 4 neurons and relu activation function
    layers.Dense(1, activation="sigmoid") # adding a dense layer with 1 neuron and sigmoid activation function
])
model.compile(optimizer="rmsprop", # compiling the model with optimizer as rmsprop
              loss="binary_crossentropy", # setting loss function as binary_crossentropy
              metrics=["accuracy"]) # setting the metrics as accuracy
history_smaller_model = model.fit( # fitting the model with train_data and train_labels
    train_data, train_labels,
    epochs=20, batch_size=512, validation_split=0.4) # setting the epochs as 20, batch size as 512 and validation split as 0.4

**Version of the model with higher capacity**

In [ ]:
model = keras.Sequential([ # creating a sequential model
    layers.Dense(512, activation="relu"), # adding a dense layer with 512 neurons and relu activation function
    layers.Dense(512, activation="relu"), # adding a dense layer with 512 neurons and relu activation function
    layers.Dense(1, activation="sigmoid") # adding a dense layer with 1 neuron and sigmoid activation function
])
model.compile(optimizer="rmsprop", # compiling the model with optimizer as rmsprop
              loss="binary_crossentropy", # setting loss function as binary_crossentropy
              metrics=["accuracy"]) # setting the metrics as accuracy
history_larger_model = model.fit( # fitting the model with train_data and train_labels
    train_data, train_labels,
    epochs=20, batch_size=512, validation_split=0.4) # setting the epochs as 20, batch size as 512 and validation split as 0.4

#### Adding weight regularization

**Adding L2 weight regularization to the model**

In [ ]:
from tensorflow.keras import regularizers # importing regularizers from keras library
model = keras.Sequential([ # creating a sequential model
    layers.Dense(16, # adding a dense layer with 16 neurons
                 kernel_regularizer=regularizers.l2(0.002), # setting the kernel_regularizer as l2 with 0.002
                 activation="relu"), # setting the activation function as relu
    layers.Dense(16, # adding a dense layer with 16 neurons
                 kernel_regularizer=regularizers.l2(0.002), # setting the kernel_regularizer as l2 with 0.002
                 activation="relu"), # setting the activation function as relu
    layers.Dense(1, activation="sigmoid") # adding a dense layer with 1 neuron and sigmoid activation function
])
model.compile(optimizer="rmsprop", # compiling the model with optimizer as rmsprop
              loss="binary_crossentropy", # setting loss function as binary_crossentropy
              metrics=["accuracy"]) # setting the metrics as accuracy
history_l2_reg = model.fit( # fitting the model with train_data and train_labels
    train_data, train_labels, 
    epochs=20, batch_size=512, validation_split=0.4) # setting the epochs as 20, batch size as 512 and validation split as 0.4

**Different weight regularizers available in Keras**

In [ ]:
from tensorflow.keras import regularizers # importing regularizers from keras library
regularizers.l1(0.001) # setting the regularizer as l1 with 0.001
regularizers.l1_l2(l1=0.001, l2=0.001) # setting the regularizer as l1_l2 with l1 as 0.001 and l2 as 0.001

#### Adding dropout

**Adding dropout to the IMDB model**

In [ ]:
model = keras.Sequential([ # creating a sequential model
    layers.Dense(16, activation="relu"), # adding a dense layer with 16 neurons and relu activation function
    layers.Dropout(0.5), # adding a dropout layer with 0.5
    layers.Dense(16, activation="relu"), # adding a dense layer with 16 neurons and relu activation function
    layers.Dropout(0.5), # adding a dropout layer with 0.5
    layers.Dense(1, activation="sigmoid") # adding a dense layer with 1 neuron and sigmoid activation function
])
model.compile(optimizer="rmsprop", # compiling the model with optimizer as rmsprop
              loss="binary_crossentropy", # setting loss function as binary_crossentropy
              metrics=["accuracy"]) # setting the metrics as accuracy
history_dropout = model.fit( # fitting the model with train_data and train_labels
    train_data, train_labels, 
    epochs=20, batch_size=512, validation_split=0.4) # setting the epochs as 20, batch size as 512 and validation split as 0.4

## Summary